!wget https://github.com/ihpar/ibu_cv/raw/main/cif_10/dataset.zip

!unzip /content/dataset.zip -d cif_10


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from PIL import Image
import sys
from collections import Counter

In [ ]:
# google colab klasör yolu
# CIF_10_KLASORU = "/content/cif_10/"

# lokal yol
CIF_10_KLASORU = "dataset/"

In [ ]:
def dosya_oku(dosya_yolu):
    with open(dosya_yolu, "rb") as dosya:
        return pickle.load(dosya, encoding="bytes")

In [ ]:
data_batch_1 = dosya_oku(CIF_10_KLASORU + "data_batch_1")
batches_meta = dosya_oku(CIF_10_KLASORU + "batches.meta")

print(data_batch_1.keys())
print(batches_meta.keys())

In [ ]:
tum_resimler = data_batch_1[b"data"]
tum_resimler.shape

In [ ]:
ilk_resim = tum_resimler[0]
ilk_resim.shape

In [ ]:
def resmi_donustur(resim):
    resim = resim.reshape(3, 32, 32)
    resim = resim.transpose(1, 2, 0)
    return resim

In [ ]:
ilk_resim = resmi_donustur(ilk_resim)

In [ ]:
ilk_resim.shape

In [ ]:
plt.imshow(ilk_resim)
plt.show()

In [ ]:
print(batches_meta[b"label_names"])
print(data_batch_1[b"labels"][:15])

In [ ]:
def resmin_sinif_ismi(resim_index):
    sinif_sayisi = data_batch_1[b"labels"][resim_index]
    sinif_ismi = batches_meta[b"label_names"][sinif_sayisi]
    return sinif_ismi.decode("ascii")

In [ ]:
print(resmin_sinif_ismi(0))

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title(resmin_sinif_ismi(i))
    plt.imshow(resmi_donustur(tum_resimler[i]))

plt.tight_layout()
plt.show()

In [ ]:
tum_resimler[0][:10]

In [ ]:
def l1_distance(resim_1, resim_2):
    fark = np.abs(resim_1.astype(int) - resim_2.astype(int))
    toplam_fark = np.sum(fark)
    return toplam_fark

In [ ]:
sys.maxsize

In [ ]:
def en_yakin_komsu_l1(resim_indeksi):
    aranan_resim = tum_resimler[resim_indeksi]
    min_fark = sys.maxsize
    en_yakin_resim = None

    for i, resim in enumerate(tum_resimler):
        if i == resim_indeksi:
            continue

        fark = l1_distance(aranan_resim, resim)
        if fark < min_fark:
            min_fark = fark
            en_yakin_resim = i

    return en_yakin_resim

In [ ]:
resim_0_en_yakini = en_yakin_komsu_l1(0)
print(resim_0_en_yakini)

In [ ]:
plt.title(resmin_sinif_ismi(resim_0_en_yakini))
plt.imshow(resmi_donustur(tum_resimler[resim_0_en_yakini]))
plt.show()

In [ ]:
plt.figure(figsize=(8, 16))

for i in range(0, 8):
    plt.subplot(8, 2, i*2+1)
    plt.title(resmin_sinif_ismi(i))
    plt.imshow(resmi_donustur(tum_resimler[i]))

    en_yakin_resim = en_yakin_komsu_l1(i)
    plt.subplot(8, 2, i*2+2)
    plt.title(resmin_sinif_ismi(en_yakin_resim))
    plt.imshow(resmi_donustur(tum_resimler[en_yakin_resim]))

plt.tight_layout()
plt.show()

In [ ]:
def l2_distance(resim_1, resim_2):
    # fark_kare = np.square(resim_1.astype(int) - resim_2.astype(int))
    # fark = np.sqrt(np.sum(fark_kare))
    fark = np.linalg.norm(resim_1.astype(int) - resim_2.astype(int))
    return fark

In [ ]:
def en_yakin_komsu_l2(resim_indeksi):
    aranan_resim = tum_resimler[resim_indeksi]
    min_fark = sys.maxsize
    en_yakin_resim = None

    for i, resim in enumerate(tum_resimler):
        if i == resim_indeksi:
            continue

        fark = l2_distance(aranan_resim, resim)
        if fark < min_fark:
            min_fark = fark
            en_yakin_resim = i

    return en_yakin_resim

In [ ]:
plt.figure(figsize=(8, 16))

for i in range(0, 8):
    plt.subplot(8, 2, i*2+1)
    plt.title(resmin_sinif_ismi(i))
    plt.imshow(resmi_donustur(tum_resimler[i]))

    en_yakin_resim = en_yakin_komsu_l2(i)
    plt.subplot(8, 2, i*2+2)
    plt.title(resmin_sinif_ismi(en_yakin_resim))
    plt.imshow(resmi_donustur(tum_resimler[en_yakin_resim]))

plt.tight_layout()
plt.show()

In [ ]:
def k_en_yakin_komsu(resim_indeksi, k):
    aranan_resim = tum_resimler[resim_indeksi]
    farklar = []

    for i, resim in enumerate(tum_resimler):
        if i == resim_indeksi:
            continue
        fark = l1_distance(aranan_resim, resim)
        farklar.append(fark)

    komsu_indeksleri = np.argsort(farklar)
    k_komsu_indeksleri = komsu_indeksleri[0:k]
    k_komsu_siniflari = [resmin_sinif_ismi(k) for k in k_komsu_indeksleri]
    cogunluk_oyu = Counter(k_komsu_siniflari).most_common(1)[0][0]

    return k_komsu_indeksleri, cogunluk_oyu

In [ ]:
def plot_KNN(sorgu_resim_idx, k=3):
    indeksler, sinif = k_en_yakin_komsu(sorgu_resim_idx, k)

    print(f"En yakın resimlerin indeksleri: {indeksler}")
    print(f"{k}-NN tahmini: {sinif}")

    plt.figure(figsize=(12, 12))

    plt.subplot(1, k+1, 1)
    plt.title("Sorgu: " + resmin_sinif_ismi(sorgu_resim_idx))
    plt.imshow(resmi_donustur(tum_resimler[sorgu_resim_idx]))

    for i, resim_indeksi in enumerate(indeksler):
        plt.subplot(1, k+1, i+2)
        plt.title(resmin_sinif_ismi(resim_indeksi))
        plt.imshow(resmi_donustur(tum_resimler[resim_indeksi]))

    plt.tight_layout()
    plt.show()

In [ ]:
plot_KNN(7, 3)

In [ ]:
test_resim = tum_resimler[5]

In [ ]:
test_resim.shape

In [ ]:
test_resim_color = test_resim.reshape(3, 32, 32).transpose(1, 2, 0)
plt.imshow(test_resim_color)
plt.show()

In [ ]:
test_resim_gray = Image.fromarray(test_resim_color).convert("L")
test_resim_gray_np = np.array(test_resim_gray)
test_resim_gray_np.shape

In [ ]:
test_resim_gray_np_flat = test_resim_gray_np.reshape(32*32)
print(test_resim_gray_np_flat.shape)
print(test_resim_gray_np_flat[:10])
print(np.min(test_resim_gray_np_flat), np.max(test_resim_gray_np_flat))

In [ ]:
plt.imshow(test_resim_gray_np, cmap="gray", vmin=0, vmax=255)
plt.show()

In [ ]:
cogunluk_oyu = Counter([1, 4, 3, 3, 4, 4, 3, 1, 3, 3]).most_common(1)[0][0]
print(cogunluk_oyu)

In [ ]:
class KNN:
    def __init__(self):
        self.images = None
        self.images_color = []
        self.images_grayscale = []
        self.images_grayscale_flat = []
        self.labels = None
        self.label_names = []

    def l1_distance(self, img_1, img_2):
        return np.sum(np.abs(img_1.astype(int) - img_2.astype(int)))

    def l2_distance(self, img_1, img_2):
        return np.linalg.norm(img_1.astype(int) - img_2.astype(int))

    def train(self, images, labels, label_names):
        self.images = images

        for img in self.images:
            color_img = img.reshape(3, 32, 32).transpose(1, 2, 0)
            self.images_color.append(color_img)

            gray_img = Image.fromarray(color_img).convert("L")
            self.images_grayscale.append(gray_img)
            self.images_grayscale_flat.append(
                np.array(gray_img).reshape(32*32))

        self.images_color = np.array(self.images_color)
        self.images_grayscale = np.array(self.images_grayscale)
        self.images_grayscale_flat = np.array(self.images_grayscale_flat)

        self.labels = labels
        for label_name in label_names:
            self.label_names.append(label_name.decode("ascii"))

    def predict(self, query_img_idx, distance_fun="L1", k=3, grayscale=False):
        dataset = self.images
        if grayscale:
            dataset = self.images_grayscale_flat

        distance_finder = self.l1_distance
        if distance_fun == "L2":
            distance_finder = self.l2_distance

        distances = []
        for i, img in enumerate(dataset):
            if i == query_img_idx:
                continue
            distances.append(distance_finder(dataset[query_img_idx], img))

        nn_indexes = np.argsort(distances)[:k]
        nn_labels = []
        for idx in nn_indexes:
            nn_labels.append(self.labels[idx])

        prediction_class = Counter(nn_labels).most_common(1)[0][0]
        prediction_label = self.label_names[prediction_class]
        return prediction_label, nn_indexes

    def plot_prediction(self, query_idx, nn_indexes, gray=False):
        dataset = self.images_color
        if gray:
            dataset = self.images_grayscale
        num_nn = len(nn_indexes)
        plt.figure(figsize=(12, 12))

        plt.subplot(1, num_nn+1, 1)
        plt.title("Sorgu: " + self.label_names[self.labels[query_idx]])
        if gray:
            plt.imshow(dataset[query_idx], cmap="gray", vmin=0, vmax=255)
        else:
            plt.imshow(dataset[query_idx])

        for i, idx in enumerate(nn_indexes):
            plt.subplot(1, num_nn+1, i+2)
            plt.title(self.label_names[self.labels[idx]])
            if gray:
                plt.imshow(dataset[idx], cmap="gray", vmin=0, vmax=255)
            else:
                plt.imshow(dataset[idx])

        plt.tight_layout()
        plt.show()

    def accuracy(self, k=3):
        prediction_results = []
        for i, img_i in enumerate(self.images):
            if (i + 1) % 250 == 0:
                print(f"At idx: {i+1}")

            if i == 1000:
                break

            distances = []

            for j, img_j in enumerate(self.images):
                if i == j:
                    continue
                distances.append(self.l1_distance(img_i, img_j))

            nn_indexes = np.argsort(distances)[:k]
            nn_classes = [self.labels[x] for x in nn_indexes]
            prediction = Counter(nn_classes).most_common(1)[0][0]
            if prediction == self.labels[i]:
                prediction_results.append(1)
            else:
                prediction_results.append(0)

        prediction_results = np.array(prediction_results)
        return prediction_results

In [ ]:
knn = KNN()
knn.train(data_batch_1[b"data"],
          data_batch_1[b"labels"],
          batches_meta[b"label_names"])

In [ ]:
print(knn.images.shape)
print(knn.images_color.shape)
print(knn.images_grayscale.shape)
print(knn.images_grayscale_flat.shape)

In [ ]:
knn.labels[:5]

In [ ]:
knn.label_names

In [ ]:
prediction_label, nn = knn.predict(7, distance_fun="L1", k=5, grayscale=False)

In [ ]:
prediction_label

In [ ]:
nn

In [ ]:
knn.plot_prediction(7, nn, gray=False)

In [ ]:
prediction_results = knn.accuracy(k=3)

In [187]:
prediction_results[:40]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.mean(prediction_results)